In [150]:
from UtilsFunc import dirlistfiles,filterlist
from glob import glob
import os 
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io, exposure
#from UtilsViz import *
from tqdm import tqdm
import pyspatialml


from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
import pickle
from sklearn.metrics import mean_squared_error, r2_score

In [151]:
pathdir = 'patch256x256'
outtiles_dir = '/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x1degree/'
tilenames = os.listdir(outtiles_dir)
i = 2
tile_patches_path = os.path.join(outtiles_dir,tilenames[i],pathdir)
tile_patches_path

'/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x1degree/N10_E105/patch256x256'

In [152]:
fs = dirlistfiles(tile_patches_path, 'tif', True)
for fi in fs:
    fo = fi.replace('..tif','.tif')
    os.rename(fi, fo)


subdirectiries included
37584


In [153]:
def files_filter_list(flist, k):
    f = [i for i in flist if k in i]
    print(len(f))
    return f

In [154]:
zdif_files = files_filter_list(fs, 'zdif')
lidar_files = files_filter_list(fs, 'LiDAR_MekongDelta_EGM08')

txd_files = files_filter_list(fs, 'TDX_DEM_EGM08')
cop_files = files_filter_list(fs, 'COP_DEM')
merit_files = files_filter_list(fs, 'MERIT_EGM08')
nasa_files = files_filter_list(fs, 'NASA_H_EGM08')
aw3d_files = files_filter_list(fs, 'AW3D_H_EGM08')

egm96_files = files_filter_list(fs, 'emg96')
s1_files = files_filter_list(fs, 'S1')
s2_files = files_filter_list(fs, 'S2')

ethm_files = files_filter_list(fs, 'ETHm')
eths_files = files_filter_list(fs, 'ETHs')
fnf_files = files_filter_list(fs, 'TDX_FNF')
esawc_files = files_filter_list(fs, 'WC')
wsf_files = files_filter_list(fs, 'WSF')

vnames = ['zdif','lidar','tdx','cop','merit','nasa','aw3d','egm96', 'ethm','eths',
          'fnf','esawc','wsf', 's1vv','s1vh','s1a','s2r','s2g','s2b']

namefiles = ['zdif','lidar','tdx','cop','merit','nasa','aw3d','egm96', 'ethm','eths',
             'fnf','esawc','wsf', 's1','s2']
print(len(namefiles))

1296
1296
1296
1296
1296
1296
1296
1296
1296
1296
1296
1296
1296
1296
1296
15


In [155]:
df = pd.DataFrame(list(zip(zdif_files,lidar_files,txd_files,
                           cop_files,merit_files,nasa_files,
                           aw3d_files,egm96_files,ethm_files,
                           eths_files,fnf_files,esawc_files,
                           wsf_files,s1_files,s2_files)),
                columns=namefiles)
#df['index'] = df.index
csv_tile_patches_path = os.path.join(tile_patches_path, f'{tilenames[i]}_patches_paths.csv')
df.to_csv(csv_tile_patches_path, index=False)

In [156]:
df

,zdif,lidar,tdx,cop,merit,nasa,aw3d,egm96,ethm,eths,fnf,esawc,wsf,s1,s2
0,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...
1,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...
2,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...
3,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...
4,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...,/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x...

## b2

In [190]:
def generate_random_indices(t,N=180):
    import numpy as np 
    np.random.seed(t.shape[0])
    if N is not None:
        N = int(N + N*0.2)
        indices = np.random.randint(1, t.shape[0], size = N)
        print(len(indices))
    else:
        indices = np.random.randint(1, t.shape[0], size = t.shape[0])
        print(len(indices))

    total_length = len(indices)
    proportions = [0.7, 0.2, 0.1]
    sublist_lengths = [int(total_length * prop) for prop in proportions]
    train = indices[:sublist_lengths[0]]
    valid = indices[sublist_lengths[0]:sublist_lengths[0]+sublist_lengths[1]]
    test = indices[sublist_lengths[0]+sublist_lengths[1]:]

    
    print('train:', len(train))
    print('valid:', len(valid))
    print('test :', len(test))

    return list(train), list(valid), list(test)

def get_idx_batch_paths(t,idx):
    one_patch_paths = t.iloc[idx:idx+1,].values[0].tolist()
    return one_patch_paths

In [188]:
path = '/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x1degree/'
fs = sorted(glob(f'{path}/*//**//*_patches_paths.csv', recursive=True)); print(len(fs))
fs[-2]

5


'/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x1degree/N10_E105/patch256x256/N10_E105_patches_paths.csv'

In [186]:
fs

['/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x1degree/N09_E105/patch256x256/N09_E105_patches_paths.csv',
 '/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x1degree/N10_E106/patch256x256/N10_E106_patches_paths.csv',
 '/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x1degree/N10_E105/patch256x256/N10_E105_patches_paths.csv',
 '/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x1degree/N10_E104/patch256x256/N10_E104_patches_paths.csv',
 '/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x1degree/N09_E106/patch256x256/N09_E106_patches_paths.csv']

In [157]:


idx = 104

In [158]:
idx_patch_paths = get_idx_batch_paths(df,idx)

In [159]:
idx_patch_paths

['/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x1degree/N10_E105/patch256x256/N10_E105_zdif/tile_1280_8192.tif',
 '/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x1degree/N10_E105/patch256x256/N10_E105_LiDAR_MekongDelta_EGM08/tile_1280_8192.tif',
 '/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x1degree/N10_E105/patch256x256/N10_E105_TDX_DEM_EGM08/tile_1280_8192.tif',
 '/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x1degree/N10_E105/patch256x256/N10_E105_COP_DEM/tile_1280_8192.tif',
 '/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x1degree/N10_E105/patch256x256/N10_E105_MERIT_EGM08/tile_1280_8192.tif',
 '/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x1degree/N10_E105/patch256x256/N10_E105_NASA_H_EGM08/tile_1280_8192.tif',
 '/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x1degree/N10_E105/patch256x256/N10_E105_AW3D_H_EGM08/tile_1280_8192.tif',
 '/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x1degree/N10_E105/patch256x256/N10_E105_emg96/tile_1280_8192.tif',
 '/media/ljp238/6tb/Joseph/DATASETS/OUT_T

In [161]:
train_indices,valid_indices,test_indices = generate_random_indices(df)

216
train: 151
valid: 43
test : 22


In [162]:
def get_idx_batch_pathslist(idx_patch_paths):
    zdif_files = files_filter_list(idx_patch_paths, 'zdif')[0]
    lidar_files = files_filter_list(idx_patch_paths, 'LiDAR_MekongDelta_EGM08')[0]

    txd_files = files_filter_list(idx_patch_paths, 'TDX_DEM_EGM08')[0]
    cop_files = files_filter_list(idx_patch_paths, 'COP_DEM')[0]
    merit_files = files_filter_list(idx_patch_paths, 'MERIT_EGM08')[0]
    nasa_files = files_filter_list(idx_patch_paths, 'NASA_H_EGM08')[0]
    aw3d_files = files_filter_list(idx_patch_paths, 'AW3D_H_EGM08')[0]

    egm96_files = files_filter_list(idx_patch_paths, 'emg96')[0]
    s1_files = files_filter_list(idx_patch_paths, 'S1')[0]
    s2_files = files_filter_list(idx_patch_paths, 'S2')[0]

    ethm_files = files_filter_list(idx_patch_paths, 'ETHm')[0]
    eths_files = files_filter_list(idx_patch_paths, 'ETHs')[0]
    fnf_files = files_filter_list(idx_patch_paths, 'TDX_FNF')[0]
    esawc_files = files_filter_list(idx_patch_paths, 'WC')[0]
    wsf_files = files_filter_list(idx_patch_paths, 'WSF')[0]

    numpaths = [zdif_files,lidar_files,txd_files,cop_files,merit_files,
                 nasa_files,aw3d_files,s2_files]#,s1_files]
    numnames = ['zdif', 'lidar', 'tdx', 'cop', 'merit', 'nasa', 'aw3d', 's2']#, 's1']
    catnames = ['ethm', 'eths', 'fnf', 'esawc', 'wsf']
    catpaths = [ethm_files,eths_files,fnf_files,esawc_files,wsf_files]
    
    return numnames, numpaths #,catnames,catpaths

def get_idx_batch_paths_names_41D(idx_patch_paths):
    zdif_files = files_filter_list(idx_patch_paths, 'zdif')[0]
    lidar_files = files_filter_list(idx_patch_paths, 'LiDAR_MekongDelta_EGM08')[0]

    txd_files = files_filter_list(idx_patch_paths, 'TDX_DEM_EGM08')[0]
    cop_files = files_filter_list(idx_patch_paths, 'COP_DEM')[0]
    merit_files = files_filter_list(idx_patch_paths, 'MERIT_EGM08')[0]
    nasa_files = files_filter_list(idx_patch_paths, 'NASA_H_EGM08')[0]
    aw3d_files = files_filter_list(idx_patch_paths, 'AW3D_H_EGM08')[0]

    egm96_files = files_filter_list(idx_patch_paths, 'emg96')[0]
    s1_files = files_filter_list(idx_patch_paths, 'S1')[0]
    s2_files = files_filter_list(idx_patch_paths, 'S2')[0]

    ethm_files = files_filter_list(idx_patch_paths, 'ETHm')[0]
    eths_files = files_filter_list(idx_patch_paths, 'ETHs')[0]
    fnf_files = files_filter_list(idx_patch_paths, 'TDX_FNF')[0]
    esawc_files = files_filter_list(idx_patch_paths, 'WC')[0]
    wsf_files = files_filter_list(idx_patch_paths, 'WSF')[0]

    numpaths = [zdif_files,lidar_files,txd_files,cop_files,merit_files,
                 nasa_files,aw3d_files,s2_files]#,s1_files]
    numnames = ['zdif', 'lidar', 'tdx', 'cop', 'merit', 'nasa', 'aw3d', 's2r','s2g','s2b']#, 's1']
    return numnames, numpaths 

In [192]:
fs = sorted(glob(f'{path}/*//**//*_patches_paths.csv', recursive=True))
csv_tile_patches_path  = fs[-2] 
csv_tile_patches_path

'/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x1degree/N10_E105/patch256x256/N10_E105_patches_paths.csv'

In [195]:
R, H = os.path.split(csv_tile_patches_path)
outproc = os.path.join(R, 'outproc')
os.makedirs(outproc, exist_ok=True)

In [206]:
idx = 23 #@
Nsize = 100 #@
t = pd.read_csv(csv_tile_patches_path) #@
outproc ='/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x1degree/N10_E105/patch256x256/outproc/' #@
train_indices,valid_indices,test_indices = generate_random_indices(t,Nsize)
nsample_dirpath = os.path.join(outproc, f'Train{Nsize}patches')
#print(nsample_dirpath)
os.makedirs(nsample_dirpath, exist_ok=True)

# read to make visualization context S2RGB, and S1 :: water mask with S1

120
train: 84
valid: 24
test : 12


In [164]:
def write_idxpatch_to_dfparquet(t,idx, nsample_dirpath):
    idx_patch_paths = get_idx_batch_paths(t,idx)
    numnames, numpaths = get_idx_batch_paths_names_41D(idx_patch_paths)
    s = pyspatialml.Raster(numpaths)
    s.names = numnames
    bname = os.path.basename(numpaths[0])[:-4] + f'_idx{idx}.parquet'
    fparquet = os.path.join(nsample_dirpath, bname)
   # fgpkg = fparquet.replace('.parquet', '.gpkg')
    ds = s.to_pandas()
    ds['id'] = str(idx)
    ds.to_parquet(fparquet, index=False)
    del s 
    print(idx)
    print(fparquet)
    return ds

def load_data_byindices(indices,t, nsample_dirpath):
    dfs = [write_idxpatch_to_dfparquet(t,idx, nsample_dirpath) for idx in indices]
    di = pd.concat(dfs)
    return di 

def split_train_valid(df, tsize=0.15):
    trainx, validx, trainy, validy = train_test_split(df.drop(['zdif'], axis=1), 
                                                  df['zdif'],
                                                  shuffle= True,
                                                  test_size=tsize)


    print(trainx.shape, trainy.shape)
    print(validx.shape, validy.shape)
    return trainx, validx, trainy, validy 
    
acols = ['x', 'y', 'zdif', 'lidar', 'tdx', 'cop', 'merit', 'nasa', 'aw3d', 's2r','s2g', 's2b', 'id']
rcv_cols = ['zdif', 'cop', 'merit', 'nasa', 'aw3d', 's2r','s2g', 's2b']

In [165]:
get_idx_batch_paths??

Signature: get_idx_batch_paths(t, idx)
Docstring: <no docstring>
Source:   
def get_idx_batch_paths(t,idx):
    one_patch_paths = t.iloc[idx:idx+1,].values[0].tolist()
    return one_patch_paths
File:      /tmp/ipykernel_1834585/3391687888.py
Type:      function

In [ ]:
traindf = load_data_byindices(train_indices,t, nsample_dirpath)
#validdf = load_data_byindices(valid_indices,numnames, numpaths, nsample_dirpath)
#testdf = load_data_byindices(test_indices,numnames, numpaths, nsample_dirpath)



In [167]:
import numpy as np 

In [168]:
df = traindf[rcv_cols]
df = df[df.zdif >= np.min(df.cop)-1] # use copernicus to filter low and high values or lidar ## apply to 2D too
df = df[df.zdif <= np.max(df.cop)+1] # 100
dx = df.copy()
dx = dx.dropna()
dx.isna().sum()

# check if this pixel being removed are truly noisy
# check the extrapolation 
# build a pipeline for copernicus : global terrain 

zdif     0
cop      0
merit    0
nasa     0
aw3d     0
s2r      0
s2g      0
s2b      0
dtype: int64

In [169]:
dx.describe().T

,count,mean,std,min,25%,50%,75%,max
zdif,45715400.0,1.254937e+00,2.153655e+00,-14.487415,0.441187,0.724891,1.528829,3.368354e+02
cop,45715400.0,3.687140e+00,1.264900e+01,-13.487518,1.903518,2.393340,3.467498,5.907065e+02
merit,45715400.0,1.339829e+32,2.135228e+35,-9.224587,2.909314,3.549895,4.230926,3.402823e+38
nasa,45715400.0,3.017763e+00,1.279840e+01,-44.380138,0.602493,2.043895,3.681893,5.895323e+02
aw3d,45715400.0,3.895888e+00,1.265014e+01,-46.227753,1.620298,2.944543,4.535709,5.792883e+02
s2r,45715400.0,6.560725e-02,3.755638e-02,-0.000707,0.043448,0.056431,0.074907,1.079650e+00
s2g,45715400.0,7.832066e-02,3.078136e-02,-0.000598,0.063260,0.071945,0.083673,1.053784e+00
s2b,45715400.0,4.612614e-02,3.151475e-02,-0.000445,0.030358,0.038408,0.050057,1.070776e+00


In [170]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler,PowerTransformer,Normalizer
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
import pickle
from sklearn.metrics import mean_squared_error, r2_score
### https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing

In [171]:
trainx, validx, trainy, validy = split_train_valid(dx)

(38858090, 7) (38858090,)
(6857310, 7) (6857310,)


In [172]:
def perf_eval(t,p):
    r2 = r2_score(t,p) # why are squared is not really good for spatial data - that lady thesis 
    rmse = mean_squared_error(t,p, squared=False)
    print(f'RMSE {rmse} and R2 {r2}')


def scale_data(xdf, method='StandardScaler'):
    if method == 'StandardScaler': scaler = StandardScaler()
    elif method == 'MinMaxScaler': scaler = MinMaxScaler()
    elif method == 'RobustScaler': scaler = RobustScaler()
    elif method == 'Normalizer': scaler = Normalizer()
    ### add more
    dfx = scaler.fit_transform(xdf)
    return dfx


def fit_catboost(trainx, validx, trainy, validy,model_path):
    ## make it generalizeble []

    early_stop = 100
    floss  = 'RMSE'
    lrate = 0.1
    model = CatBoostRegressor(iterations=10000, verbose=100, 
                                early_stopping_rounds=early_stop, 
                                loss_function= floss,
                                depth = 6, 
                                learning_rate=lrate,
                                task_type="GPU", devices="0:1")
    model.fit(trainx, trainy, eval_set=(validx, validy))

    pickle_write_model(model, model_path)
    return model

def pickle_load_model(model_path):
    pickled_model = pickle.load(open(model_path, 'rb'))
    return pickled_model

def pickle_write_model(model, model_path):
    pickle.dump(model, open(model_path, 'wb'))
    return model_path

In [173]:
trainx_s = scale_data(trainx, 'StandardScaler')
trainx_m = scale_data(trainx, 'MinMaxScaler')
trainx_r = scale_data(trainx, 'RobustScaler')
trainx_n = scale_data(trainx, 'Normalizer')

validx_s = scale_data(validx, 'StandardScaler')
validx_m = scale_data(validx, 'MinMaxScaler')
validx_r = scale_data(validx, 'RobustScaler')
validx_n = scale_data(validx, 'Normalizer')

In [224]:
import datetime
import pytz

def get_current_datetime_uk():
    # Set the timezone to UK
    uk_tz = pytz.timezone('Europe/London')

    # Get the current datetime in UK timezone
    now = datetime.datetime.now(uk_tz)

    # Format the datetime as a string
    date_str = now.strftime('%Y-%m-%d %H:%M:%S %Z%z')

    # Return the date and time as a list
    a = date_str.split()
    dt = f'{a[0]}_{a[1]}_{a[2]}'.replace(':','-')
    return dt

# Call the function and print the result
print(get_current_datetime_uk())


2023-03-13_16-18-52_GMT+0000


In [218]:
a = get_current_datetime_uk()

In [219]:
a

['2023-03-13', '16:16:12', 'GMT+0000']

In [223]:
dt = f'{a[0]}_{a[1]}_{a[2]}'.replace(':','-')
dt 

'2023-03-13_16-16-12_GMT+0000'

In [ ]:
modeldir = os.path.join(nsample_dirpath, 'models')
os.makedirs(modeldir, exist_ok=True)
modelpath_s = os.path.join(modeldir, f'cboost_s_{Nsize}.pkl')
modelpath_m = os.path.join(modeldir, f'cboost_m_{Nsize}.pkl')
modelpath_r = os.path.join(modeldir, f'cboost_r_{Nsize}.pkl')
modelpath_n = os.path.join(modeldir, f'cboost_n_{Nsize}.pkl')


model_s = fit_catboost(trainx_s, validx_s, trainy, validy,modelpath_s)
model_m = fit_catboost(trainx_m, validx_m, trainy, validy,modelpath_m)
model_r = fit_catboost(trainx_r, validx_r, trainy, validy,modelpath_r)
model_n = fit_catboost(trainx_n, validx_n, trainy, validy,modelpath_n)

In [175]:
def eval_model(model, X,y):
    p = model.predict(X)
    perf_eval(y,p)


#plot history 

In [176]:
models = [model_s,model_m,model_r,model_n]
eval_model(model_s, trainx_s, trainy)
eval_model(model_m, trainx_m, trainy)
eval_model(model_r, trainx_r, trainy)
eval_model(model_n, trainx_n, trainy)

print('')

eval_model(model_s, validx_s, validy)
eval_model(model_m, validx_m, validy)
eval_model(model_r, validx_r, validy)
eval_model(model_n, validx_n, validy)
# write this into a text file and save same place like the modelpath

RMSE 1.7519690993515324 and R2 0.3382839287245234
RMSE 1.7457869807104265 and R2 0.3429456430309683
RMSE 1.7126043870049692 and R2 0.36768584647921976
RMSE 1.7543400105909004 and R2 0.3364917377156843

RMSE 1.7629571510970568 and R2 0.3296587143204204
RMSE 1.7785528057161655 and R2 0.31774616997900385
RMSE 1.7240076512801874 and R2 0.3589515828765949
RMSE 1.7660481592759418 and R2 0.3273060231618292


In [225]:
ad = {'R^2': 0.32561376344981496, 'MAE': 0.8185511826709, 'MSE': 2.713837073442267, 'RMSE': 1.6473727791372137, 'Explained Variance Score': 0.32561416611396155, 'MeAE': 0.2901570536849505}


In [226]:
ad

{'R^2': 0.32561376344981496,
 'MAE': 0.8185511826709,
 'MSE': 2.713837073442267,
 'RMSE': 1.6473727791372137,
 'Explained Variance Score': 0.32561416611396155,
 'MeAE': 0.2901570536849505}

In [177]:
# even for 2d remove pixel vals that don't make sense too low/high

In [178]:
#trainx.describe()

In [179]:
#pd.DataFrame(trainx_s).describe()

In [180]:
#pd.DataFrame(trainx_m).describe()

In [181]:
#pd.DataFrame(trainx_r).describe()

In [182]:
#pd.DataFrame(trainx_n).describe()

In [183]:
traindf.columns

Index(['x', 'y', 'zdif', 'lidar', 'tdx', 'cop', 'merit', 'nasa', 'aw3d', 's2r',
       's2g', 's2b', 'id'],
      dtype='object')

In [184]:
plotS2RGB(numpaths[-2])

NameError: name 'plotS2RGB' is not defined

In [ ]:
# where's that script that create gkpg for sampling patches and Id???
### look @2baseline 

/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x1degree/N10_E105/patch256x256/outproc/Nsize10patches
12
train: 8
valid: 2
test : 2
